# LangChain流式处理

由于大部分任务LLM没有办法立即给出输出结果，所谓为了更好的用体验，在处理和用户交互任务或者日志打印进度时，需要考虑流式处理。

## 支持的流式处理的模式

1. values： 每个步骤之后都会打印完整值
2. updates: 只输出在节点中state有变化的值
3. custom: 从图形节点内部流式传输自定义数据
4. messages: 只输出在节点中调用LLM
5. debug: 打印每个节点更详细的信息

## 代理进度

In [ ]:
from dotenv import load_dotenv
load_dotenv()

from langchain.tools import tool
@tool
def get_weather(city:str) -> str:
    """Get weather for a given city."""

    return f"It's always sunny in {city}!"

from langchain_deepseek import ChatDeepSeek
llm = ChatDeepSeek(
    model="deepseek-chat"
)
from langchain.agents import create_agent
agent = create_agent(
    model=llm,
    tools=[get_weather]
)
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode="updates"
):
    for step, data in chunk.items():
        print(f"step: {step}")
        print(f"content: {data['messages'][-1].content_blocks}")

## LLM Tokens

要流式传输 LLM 生成的令牌，请使用 stream_mode="messages"。下面您可以看到代理流式传输工具调用和最终响应的输出。

In [ ]:
from dotenv import load_dotenv
load_dotenv()

from langchain.tools import tool
@tool
def get_weather(city:str) -> str:
    """Get weather for a given city."""

    return f"It's always sunny in {city}!"

from langchain_deepseek import ChatDeepSeek
llm = ChatDeepSeek(
    model="deepseek-chat"
)
from langchain.agents import create_agent
agent = create_agent(
    model=llm,
    tools=[get_weather]
)
for token, metadata in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode="messages"
):
    print(f"node: {metadata['langgraph_node']}")
    print(f"content: {token.content_blocks}")
    print("\n")

## 自定义更新

In [ ]:
from dotenv import load_dotenv
load_dotenv()

from langchain.tools import tool
from langgraph.config import get_stream_writer  
@tool
def get_weather(city:str) -> str:
    """Get weather for a given city."""
    writer = get_stream_writer()  
    # stream any arbitrary data
    writer(f"Looking up data for city: {city}")
    writer(f"Acquired data for city: {city}")
    return f"It's always sunny in {city}!"

from langchain_deepseek import ChatDeepSeek
llm = ChatDeepSeek(
    model="deepseek-chat"
)
from langchain.agents import create_agent
agent = create_agent(
    model=llm,
    tools=[get_weather]
)
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode="custom"
):
    print(chunk)

## 流式传输多种模式

您可以通过将流模式作为列表传递来指定多个流模式：stream_mode=["updates", "custom"]。

流式传输的输出将是 (mode, chunk) 元组，其中 mode 是流模式的名称，而 chunk 是该模式流式传输的数据。

In [ ]:
from dotenv import load_dotenv
load_dotenv()

from langchain.tools import tool
from langgraph.config import get_stream_writer  
@tool
def get_weather(city:str) -> str:
    """Get weather for a given city."""
    writer = get_stream_writer()  
    # stream any arbitrary data
    writer(f"Looking up data for city: {city}")
    writer(f"Acquired data for city: {city}")
    return f"It's always sunny in {city}!"

from langchain_deepseek import ChatDeepSeek
llm = ChatDeepSeek(
    model="deepseek-chat"
)
from langchain.agents import create_agent
agent = create_agent(
    model=llm,
    tools=[get_weather]
)
for stream_mode, chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["updates", "custom"]
):
    print(f"stream_mode: {stream_mode}")
    print(f"content: {chunk}")
    print("\n")